# Part 1

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

- Data from square 1 is carried 0 steps, since it's at the access port.
- Data from square 12 is carried 3 steps, such as: down, left, left.
- Data from square 23 is carried only 2 steps: up twice.
- Data from square 1024 must be carried 31 steps.
- How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

Your puzzle input is 361527.

In [3]:
import itertools

In [10]:
def num_steps(start):
    steps = 0
    if start == 1:
        return steps
    
    # first find which square we're on, 3x3, 5x5, etc.
    for i, side in enumerate(itertools.count(3, 2), start=1):
        if start <= side ** 2:
            sides = side
            steps += i
            break
    
    # figure out how far to the middle of this side of the square
    highest_middle = sides ** 2 - (sides // 2)
    middles = [highest_middle - (sides - 1) * i for i in range(4)]
    closest_middle = min(middles, key=lambda x: abs(x - start))
    steps += abs(start - closest_middle)
    
    return steps

In [11]:
assert num_steps(1) == 0
assert num_steps(12) == 3
assert num_steps(23) == 2, num_steps(23)
assert num_steps(1024) == 31, num_steps(1024)

In [12]:
num_steps(361527)

326

# Part 2

As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

- Square 1 starts with the value 1.
- Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
- Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
- Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
- Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.
- Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```

What is the first value written that is larger than your puzzle input?

In [23]:
import itertools
from collections import defaultdict

import numpy as np

In [53]:
directions = {
    'up': np.array([0, 1]),
    'down': np.array([0, -1]),
    'left': np.array([-1, 0]),
    'right': np.array([1, 0])
}

turns = {
    'right': 'up',
    'up': 'left',
    'left': 'down',
    'down': 'right'
}

def grid_path():
    pos = np.array([0, 0])
    direction = 'right'
    num_steps = 1
    step_counter = 0
    sides_traversed = 0
    
    while True:
        yield tuple(pos)
        
        pos = pos + directions[direction]
        step_counter += 1
        
        # time for a turn
        if step_counter == num_steps:
            direction = turns[direction]
            step_counter = 0
            sides_traversed += 1
            
            # every 2 turns increase the number of steps taken
            if sides_traversed == 2:
                num_steps += 1
                sides_traversed = 0

In [31]:
list(itertools.islice(grid_path(), 10))

[(0, 0),
 (1, 0),
 (1, 1),
 (0, 1),
 (-1, 1),
 (-1, 0),
 (-1, -1),
 (0, -1),
 (1, -1),
 (2, -1)]

In [49]:
surroundings = list(itertools.product([-1, 0, 1], [-1, 0, 1]))
surroundings.remove((0, 0))

def spiral_grid_accum(pzl_input):
    grid = defaultdict(lambda: 0)
    grid[(0, 0)] = 1
    
    for pos in grid_path():
        if grid[pos] > pzl_input:
            return grid[pos]
    
        for x in surroundings:
            grid[(tuple(np.add(pos, x)))] += grid[pos]

In [46]:
surroundings

[(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

In [51]:
assert spiral_grid_accum(4) == 5, spiral_grid_accum(4)
assert spiral_grid_accum(100) == 122, spiral_grid_accum(100)

In [52]:
spiral_grid_accum(361527)

363010